In [80]:
import skimage as ski
import skimage.io
import numpy as np

from pydash import py_
from pprint import pprint, pformat

from scipy import ndimage as nd

In [51]:
nyan = ski.io.imread('/d/nyan.png', as_gray=True)


In [74]:
def bin_pixel_value(value, depth=2):
  '''Bin a pixel value to the specified bits'''
  
  if value < 0.1:
    return 0b00
  elif value < 0.6:
    return 0b01
  elif value < 0.9:
    return 0b10
  else:
    return 0b11

def compose_byte(pxs):
  # pxs is the 4 pixel chunk from flat frame
  composed = 0
  for i, px in enumerate(pxs):
    composed = (composed << 2) | bin_pixel_value(px)

  return composed

  
def encode_frame(frame):
  '''Encode a frame
  The encoding scheme was described in `decompose_frame`. This function
  does the reverse.
  
  To properly encode a greyscale image, we would need to replace pixel values
  using a lookup table.
  
  Since we have two possible image schemes, at 2bits and 1bit, there'll be either
  4 or 8 pixel data packed into single frame.
  
  For 1bit images, the scheme is pretty simple: We use a binary image.
  Given a 2D image buffer, we stack the rows horizontally and then take 8 elements
  from the beginning and pack.
  
  The MSB in the packed byte is the first element, and so on.
  
  For 2 bit images, we define a simple lookup table for pixel values:
  
  `0b00` if value < 0.1
  `0b01` if value < 0.6
  `0b10` if value < 0.9
  `0b11` otherwise.
  

  ┌───┐ ┌───┬───┐┌───┬───┐┌───┬───┐┌───┬───┐
  │ 1 │ │   │   ││   │   ││   │   ││   │   │
  └───┘ └───┴───┘└───┴───┘└───┴───┘└───┴───┘
   DC   ─ 7 ─── 8 Bits Transmission ──── 0 ─ 

        ─────────                            Px (n-4)
                 ─────────                   Px (n-3)
                          ─────────          Px (n-2)
                                   ───────── Px (n-1)

  '''
  # XXX: We have magic numbers here!
  framebuffer = np.zeros(120 * 120)
  
  # Flatten the frame and create a 
  fgrps = np.split(frame.ravel(), 14400)
  
  return [compose_byte(x) for x in fgrps]

In [123]:
def serialize_frame(uid, image):
  serialized = (py_(image)
    .thru(encode_frame)
    .map(lambda byte: f'0x{byte:02x}')
    .thru(lambda x: pformat(x, compact=True, indent=2))
    .replace('[', ' ')
    .replace(']', ',')
    .replace('\'', '')
    .value())

  key = py_(uid).snake_case().to_upper().value()
  
  declaration = f'''
unsigned short BMP_{key}[] = {{
{serialized}
}};
  '''
  
  return declaration


header_fmt = '''
#ifndef __BITMAP_H__
#define __BITMAP_H__

{payload}

#endif
'''

imgs = {
  'nyan_r': ski.io.imread('/d/nyan.png', as_gray=True),
  'woof': ski.io.imread('/d/woofirl.png', as_gray=True),
  'nyan_l': ski.io.imread('/d/nyanl.png', as_gray=True),
  'french': ski.io.imread('/d/franc.png', as_gray=True),
}


In [124]:
encoded = [serialize_frame(k, v) for k, v in imgs.items()]

payload = '\n'.join(encoded)

with open('/r/betty-epd/firmware/bitmap.h', 'w') as fp:
  fp.write(header_fmt.format(payload=payload))


unsigned short BMP_NYAN_R = {
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff, 0xff,
  0xff, 0xff, 0xf